Load training data

In [ ]:
import pandas as pd
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
# The path of the dataset

df=pd.read_csv('/content/training_dataset.csv',sep=';')
#Print data
df.head()

,DP_ID,PORTAL,Date,Time,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW
0,543,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,19.34,10.0
1,539,"E4S 56,160",20210601,04:00:00,240,48,16,8,4,19.64,9.0
2,534,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,23.36,5.0
3,351,"E4S 56,490",20210601,04:00:00,240,48,16,8,4,20.14,7.0
4,353,"E4S 57,055",20210601,04:00:00,240,48,16,8,4,17.70,7.0


In [ ]:
df.describe()

,DP_ID,Date,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW
count,1.921967e+06,1.921967e+06,1.921967e+06,1.921967e+06,1.921967e+06,1.921967e+06,1.921967e+06,1.921967e+06,1.921905e+06
mean,2.391436e+03,2.021091e+07,4.298932e+02,8.557945e+01,2.819306e+01,1.384524e+01,6.669162e+00,2.002764e+01,1.236552e+01
std,1.891637e+03,2.010941e+02,1.017515e+02,2.035077e+01,6.780857e+00,3.384322e+00,1.678570e+00,3.027135e+00,8.023255e+00
min,1.530000e+02,2.021060e+07,2.400000e+02,4.800000e+01,1.600000e+01,8.000000e+00,4.000000e+00,0.000000e+00,1.000000e+00
25%,5.430000e+02,2.021072e+07,3.450000e+02,6.900000e+01,2.300000e+01,1.100000e+01,5.000000e+00,1.809000e+01,5.000000e+00
50%,1.254000e+03,2.021092e+07,4.330000e+02,8.600000e+01,2.800000e+01,1.400000e+01,7.000000e+00,1.985000e+01,1.200000e+01
75%,4.437000e+03,2.021111e+07,5.180000e+02,1.030000e+02,3.400000e+01,1.700000e+01,8.000000e+00,2.194000e+01,1.900000e+01
max,4.873000e+03,2.021123e+07,6.000000e+02,1.200000e+02,4.000000e+01,2.000000e+01,1.000000e+01,4.583000e+01,4.500000e+01


In [ ]:
#combine date and time and add a column to use on x-axis
import datetime
df['datetime'] = pd.to_datetime(df['Date'].astype(str)+ ' ' + df['Time'], format='%Y%m%d %H:%M:%S')
df.head()

,DP_ID,PORTAL,Date,Time,Interval_1,Interval_5,Interval_15,Interval_30,Interval_60,SPEED_MS_AVG,FLOW,datetime
0,543,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,19.34,10.0,2021-06-01 04:00:00
1,539,"E4S 56,160",20210601,04:00:00,240,48,16,8,4,19.64,9.0,2021-06-01 04:00:00
2,534,"E4S 56,780",20210601,04:00:00,240,48,16,8,4,23.36,5.0,2021-06-01 04:00:00
3,351,"E4S 56,490",20210601,04:00:00,240,48,16,8,4,20.14,7.0,2021-06-01 04:00:00
4,353,"E4S 57,055",20210601,04:00:00,240,48,16,8,4,17.70,7.0,2021-06-01 04:00:00


In [ ]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()


result = df.dropna().groupby(['PORTAL', 'Date', 'Interval_1']).agg(
    FLOW_SUM=('FLOW', 'sum'),  # Summing FLOW
    SPEED_WEIGHTED_AVG=('SPEED_MS_AVG', lambda x: weighted_average(df.loc[x.index], 'SPEED_MS_AVG', 'FLOW'))  # Applying weighted average
)

print(result)

                                FLOW_SUM  SPEED_WEIGHTED_AVG
PORTAL     Date     Interval_1                              
E4S 55,620 20210601 240             29.0           21.683103
                    241             26.0           21.933077
                    242             38.0           21.563684
                    243             34.0           22.681176
                    244             29.0           23.126552
...                                  ...                 ...
E4S 58,140 20211231 596             19.0           21.507895
                    597             28.0           20.806071
                    598             23.0           21.352609
                    599             19.0           21.491579
                    600             20.0           21.429000

[604026 rows x 2 columns]


In [ ]:
# Step 2: Reset the index so that 'PORTAL', 'Date', and 'Interval_1' are regular columns
result2 = result.reset_index()

# Check if 'PORTAL' is present after resetting the index
print(result2.columns)  # This should include 'PORTAL', 'Date', 'Interval_1', 'FLOW_SUM', 'SPEED_WEIGHTED_AVG'

# Step 3: Add a column for the sum of the next 15 Interval_1 flow values, excluding the current row
result2['FLOW_NEXT_15_SUM'] = result2.groupby(['PORTAL', 'Date'])['FLOW_SUM'].transform(
    lambda x: x.shift(+1).rolling(window=15, min_periods=15).sum().shift(-16)
)

# Step 4: Filter by the specific 'PORTAL' value (e.g., 'E4S 55,620')
rslt_df = result2[result2['PORTAL'] == 'E4S 55,620']
rslt_df2 = rslt_df[rslt_df['Date'] == 20210601]

print(rslt_df2.tail(20))

Index(['PORTAL', 'Date', 'Interval_1', 'FLOW_SUM', 'SPEED_WEIGHTED_AVG'], dtype='object')
         PORTAL      Date  Interval_1  FLOW_SUM  SPEED_WEIGHTED_AVG  \
341  E4S 55,620  20210601         581      55.0           19.102727   
342  E4S 55,620  20210601         582      59.0           18.117966   
343  E4S 55,620  20210601         583      67.0           18.572537   
344  E4S 55,620  20210601         584      49.0           19.145918   
345  E4S 55,620  20210601         585      61.0           19.509180   
346  E4S 55,620  20210601         586      46.0           19.290652   
347  E4S 55,620  20210601         587      59.0           18.776441   
348  E4S 55,620  20210601         588      56.0           18.140357   
349  E4S 55,620  20210601         589      53.0           19.276415   
350  E4S 55,620  20210601         590      58.0           17.816897   
351  E4S 55,620  20210601         591      63.0           18.506667   
352  E4S 55,620  20210601         592      70.0           

In [ ]:
result3 = result2

result3['FLOW_PREV_5_SUM'] = result3.groupby(['PORTAL', 'Date'])['FLOW_SUM'].transform(
    lambda x: x.rolling(window=5, min_periods=5).sum()
)

result3['FLOW_PREV_15_SUM'] = result3.groupby(['PORTAL', 'Date'])['FLOW_SUM'].transform(
    lambda x: x.rolling(window=15, min_periods=15).sum()
)

result3['FLOW_PREV_60_SUM'] = result3.groupby(['PORTAL', 'Date'])['FLOW_SUM'].transform(
    lambda x: x.rolling(window=60, min_periods=60).sum()
)

# Step 4: Filter by the specific 'PORTAL' value (e.g., 'E4S 55,620')
rslt_df = result3[result3['PORTAL'] == 'E4S 55,620']
rslt_df2 = rslt_df[rslt_df['Date'] == 20210601]

print(rslt_df2.head(61))

        PORTAL      Date  Interval_1  FLOW_SUM  SPEED_WEIGHTED_AVG  \
0   E4S 55,620  20210601         240      29.0           21.683103   
1   E4S 55,620  20210601         241      26.0           21.933077   
2   E4S 55,620  20210601         242      38.0           21.563684   
3   E4S 55,620  20210601         243      34.0           22.681176   
4   E4S 55,620  20210601         244      29.0           23.126552   
..         ...       ...         ...       ...                 ...   
56  E4S 55,620  20210601         296      47.0           21.021489   
57  E4S 55,620  20210601         297      53.0           20.338868   
58  E4S 55,620  20210601         298      62.0           19.848710   
59  E4S 55,620  20210601         299      46.0           21.201522   
60  E4S 55,620  20210601         300      52.0           20.211538   

    FLOW_NEXT_15_SUM  FLOW_PREV_5_SUM  FLOW_PREV_15_SUM  FLOW_PREV_60_SUM  
0              543.0              NaN               NaN               NaN  
1      

In [ ]:
result4 = result3

import pandas as pd

def weighted_average(values, weights):
    """Compute the weighted average for a given window."""
    return (values * weights).sum() / weights.sum()

# Assuming result2 already has the 'SPEED_WEIGHTED_AVG' and 'FLOW_SUM' columns
# We need to calculate the weighted average for the next 15 intervals of SPEED_WEIGHTED_AVG, weighted by FLOW_SUM

result4['SPEED_NEXT_15_AVG'] = result4.groupby(['PORTAL', 'Date']).apply(
    lambda group: group['SPEED_WEIGHTED_AVG'].shift(+1).rolling(window=15, min_periods=15).apply(
        lambda x: weighted_average(x, group['FLOW_SUM'].shift(+1).loc[x.index])  # Weighted average over rolling window
    ).shift(-16)  # Shift result back up to align with the current row
).reset_index(level=[0, 1], drop=True)

# Step 4: Filter by the specific 'PORTAL' value (e.g., 'E4S 55,620') and Date (e.g., 20210601)
rslt_df = result4[result4['PORTAL'] == 'E4S 55,620']
rslt_df2 = rslt_df[rslt_df['Date'] == 20210601]

print(rslt_df2.head(20))


        PORTAL      Date  Interval_1  FLOW_SUM  SPEED_WEIGHTED_AVG  \
0   E4S 55,620  20210601         240      29.0           21.683103   
1   E4S 55,620  20210601         241      26.0           21.933077   
2   E4S 55,620  20210601         242      38.0           21.563684   
3   E4S 55,620  20210601         243      34.0           22.681176   
4   E4S 55,620  20210601         244      29.0           23.126552   
5   E4S 55,620  20210601         245      27.0           22.453333   
6   E4S 55,620  20210601         246      38.0           21.143684   
7   E4S 55,620  20210601         247      40.0           21.339250   
8   E4S 55,620  20210601         248      31.0           22.322581   
9   E4S 55,620  20210601         249      41.0           22.172927   
10  E4S 55,620  20210601         250      31.0           22.970968   
11  E4S 55,620  20210601         251      36.0           23.240556   
12  E4S 55,620  20210601         252      36.0           22.824444   
13  E4S 55,620  2021

<ipython-input-23-f5f37639dde6>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result4['SPEED_NEXT_15_AVG'] = result4.groupby(['PORTAL', 'Date']).apply(


In [ ]:
result5 = result4

import pandas as pd

def weighted_average(values, weights):
    """Compute the weighted average for a given window."""
    return (values * weights).sum() / weights.sum()

# Assuming result2 already has the 'SPEED_WEIGHTED_AVG' and 'FLOW_SUM' columns
# We need to calculate the weighted average for the next 15 intervals of SPEED_WEIGHTED_AVG, weighted by FLOW_SUM

# Calculate the weighted average for the previous 15 intervals (including the current one) of SPEED_WEIGHTED_AVG, weighted by FLOW_SUM
result5['SPEED_PREV_5_AVG'] = result5.groupby(['PORTAL', 'Date']).apply(
    lambda group: group['SPEED_WEIGHTED_AVG'].rolling(window=5, min_periods=5).apply(
        lambda x: weighted_average(x, group['FLOW_SUM'].loc[x.index])  # Weighted average over rolling window
    )
).reset_index(level=[0, 1], drop=True)

result5['SPEED_PREV_15_AVG'] = result5.groupby(['PORTAL', 'Date']).apply(
    lambda group: group['SPEED_WEIGHTED_AVG'].rolling(window=15, min_periods=15).apply(
        lambda x: weighted_average(x, group['FLOW_SUM'].loc[x.index])  # Weighted average over rolling window
    )
).reset_index(level=[0, 1], drop=True)

result5['SPEED_PREV_60_AVG'] = result5.groupby(['PORTAL', 'Date']).apply(
    lambda group: group['SPEED_WEIGHTED_AVG'].rolling(window=60, min_periods=60).apply(
        lambda x: weighted_average(x, group['FLOW_SUM'].loc[x.index])  # Weighted average over rolling window
    )
).reset_index(level=[0, 1], drop=True)

# Step 4: Filter by the specific 'PORTAL' value (e.g., 'E4S 55,620') and Date (e.g., 20210601)
rslt_df = result4[result4['PORTAL'] == 'E4S 55,620']
rslt_df2 = rslt_df[rslt_df['Date'] == 20210601]

print(rslt_df2.head(20))


<ipython-input-24-2cfe5fb7694e>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result5['SPEED_PREV_5_AVG'] = result5.groupby(['PORTAL', 'Date']).apply(


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import holidays

result6 = result5

# Define the country for holidays (e.g., 'US' for United States)
swe_holidays = holidays.SE()

# Step 1: Ensure that 'Date' is in datetime format
result6['Date'] = pd.to_datetime(result6['Date'], format='%Y%m%d')

# Step 2: Create a dummy column based on whether the date is a weekday or a holiday
result6['IS_WEEKDAY'] = result6['Date'].apply(
    lambda date: 1 if date.weekday() < 5 and date not in swe_holidays else 0
)

# Step 3: Filter by the specific 'PORTAL' value (e.g., 'E4S 55,620') and Date (e.g., 20210601)
rslt_df = result6[result6['PORTAL'] == 'E4S 55,620']
rslt_df2 = rslt_df[rslt_df['Date'] == pd.to_datetime(20210601, format='%Y%m%d')]

# Display the result, including the new dummy variable
print(rslt_df2.head(20))


In [ ]:
result7 = pd.read_csv('/content/evalpreproV2.csv',sep=';')
#Print data
result7['Interval_30'] = (result7['Interval_1']) // 30
result7 = result7.drop(columns='IS_WEEKDAY_OR_HOLIDAY');

result7.head()

,PORTAL,Date,Interval_1,FLOW_SUM,SPEED_WEIGHTED_AVG,FLOW_NEXT_15_SUM,FLOW_PREV_5_SUM,FLOW_PREV_15_SUM,FLOW_PREV_60_SUM,SPEED_NEXT_15_AVG,SPEED_PREV_5_AVG,SPEED_PREV_15_AVG,SPEED_PREV_60_AVG,IS_WEEKDAY,Interval_30
0,"E4S 55,620",2022-01-05,240,7.0,20.242857,111.0,NaN,NaN,NaN,21.036216,NaN,NaN,NaN,1,8
1,"E4S 55,620",2022-01-05,241,3.0,21.000000,114.0,NaN,NaN,NaN,20.899211,NaN,NaN,NaN,1,8
2,"E4S 55,620",2022-01-05,242,3.0,20.100000,119.0,NaN,NaN,NaN,20.891849,NaN,NaN,NaN,1,8
3,"E4S 55,620",2022-01-05,243,3.0,21.466667,124.0,NaN,NaN,NaN,20.885242,NaN,NaN,NaN,1,8
4,"E4S 55,620",2022-01-05,244,6.0,20.920000,131.0,22.0,NaN,NaN,20.931450,20.678182,NaN,NaN,1,8


In [ ]:
file_name = 'evalpreproV3.csv'
result7.to_csv(file_name, sep=';', encoding='utf-8', index=False, header=True)